### Company X Bias Analysis

#### Imports

In [277]:
#Standard
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#AIF360 Dataset Classes 
from aif360.datasets import StandardDataset
from aif360.datasets import BinaryLabelDataset

#AIF360 Metrics Classes  
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.metrics import ClassificationMetric

#AIF360 Algorithms 
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.inprocessing import AdversarialDebiasing
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing
# AI360 Explainers
from aif360.explainers import MetricTextExplainer

#Sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.model_selection import GridSearchCV, train_test_split

#Tensorflow -- install version before 2.0
import tensorflow as tf 

#For Markdown
from IPython.display import Markdown, display

#Helper Functions
from src.classifier_functions import *

#### Load Data


In [278]:
data = pd.read_csv('../company_x.csv', index_col='employee_id')

In [279]:
data.head()


,signing_bonus,salary,degree_level,sex,yrs_experience,dept,is_manager,direct_reports,boss_id,total_reports,company_level
employee_id,,,,,,,,,,,
138719,0,273000.0,Master,M,2,engineering,False,0.0,43602,0,0
3192,0,301000.0,Bachelor,F,1,sales,False,0.0,87847,0,0
114657,0,261000.0,Master,F,2,sales,False,0.0,180854,0,0
29039,0,86000.0,High_School,F,4,HR,False,0.0,88370,0,0
118607,0,126000.0,Bachelor,F,3,sales,False,0.0,23565,0,0


In [280]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 138719 to 72227
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   signing_bonus   10000 non-null  int64  
 1   salary          10000 non-null  float64
 2   degree_level    10000 non-null  object 
 3   sex             10000 non-null  object 
 4   yrs_experience  10000 non-null  int64  
 5   dept            10000 non-null  object 
 6   is_manager      10000 non-null  bool   
 7   direct_reports  10000 non-null  float64
 8   boss_id         10000 non-null  int64  
 9   total_reports   10000 non-null  int64  
 10  company_level   10000 non-null  int64  
dtypes: bool(1), float64(2), int64(5), object(3)
memory usage: 869.1+ KB


In [281]:
## Create Label 
data_with_label = data.copy()
data_with_label['salary'] = data_with_label['salary'].transform(lambda x: x > 150000).astype(int)
data_with_label['sex'] = data_with_label['sex'].transform(lambda x: x == 'M').astype(int)
data_with_label.head()

,signing_bonus,salary,degree_level,sex,yrs_experience,dept,is_manager,direct_reports,boss_id,total_reports,company_level
employee_id,,,,,,,,,,,
138719,0,1,Master,1,2,engineering,False,0.0,43602,0,0
3192,0,1,Bachelor,0,1,sales,False,0.0,87847,0,0
114657,0,1,Master,0,2,sales,False,0.0,180854,0,0
29039,0,0,High_School,0,4,HR,False,0.0,88370,0,0
118607,0,0,Bachelor,0,3,sales,False,0.0,23565,0,0


In [282]:
### Create StandardDataset

std_data = StandardDataset(df=data_with_label,   
                         label_name='salary',
                         favorable_classes =[1],
                        protected_attribute_names=['sex'], 
                         privileged_classes=[[1]],
                        categorical_features=['degree_level', 'dept'], 
                          features_to_drop=['boss_id', 'is_manager'])



In [283]:
df_data = std_data.convert_to_dataframe()
binary_data = BinaryLabelDataset(favorable_label=1, unfavorable_label=0, df=df_data[0], label_names=['salary'],
              protected_attribute_names=['sex'])

privileged_groups= [{'sex':1}]
unprivileged_groups= [{'sex': 0}]

In [284]:
## Splitting the Data
data_train, data_vt = binary_data.split([0.7], shuffle=True)

data_val, data_test = data_vt.split([0.5], shuffle=True)

In [285]:
### First Metrics
metric_train = BinaryLabelDatasetMetric(data_train, 
                            unprivileged_groups=unprivileged_groups, 
                            privileged_groups=privileged_groups)
#Explainer 
ex_metric_train = MetricTextExplainer(metric_train)

print(ex_metric_train.mean_difference())
print('\n')
print(ex_metric_train.disparate_impact())

Mean difference (mean label value on privileged instances - mean label value on unprivileged instances): -0.1539438514958028


Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.7694434250764527


### Random Forest Classifier 

In [286]:
#Creating X,y
X_train = data_train.features
y_train = data_train.labels.ravel()
X_test = data_val.features
y_test = data_val.labels.ravel()

In [287]:
# Creating Random Forest Classifier
rfc = RandomForestClassifier(n_estimators=1000, max_depth=5)
rfc.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [288]:
#Creating Predicted Dataframe
y_pred = rfc.predict(X_test)
data_val_pred = data_val.copy()
data_val_pred.labels = y_pred.ravel()

In [326]:
df = data_val.convert_to_dataframe()[0]

df.head()

aeq_df = pd.DataFrame()

aeq_df['score'] = y_pred

aeq_df['label_value'] = df['salary'].values
aeq_df['attribute_1'] = df['sex'].values

aeq_df.to_csv('../aeq.csv', index=False)

In [310]:
rfc.score(X_test, y_test)

In [290]:
rfc.predict_proba(X_test)[:10]

array([[0.3188316 , 0.6811684 ],
       [0.8918013 , 0.1081987 ],
       [0.207084  , 0.792916  ],
       [0.33393571, 0.66606429],
       [0.32045447, 0.67954553],
       [0.32809142, 0.67190858],
       [0.3344556 , 0.6655444 ],
       [0.20656722, 0.79343278],
       [0.34651684, 0.65348316],
       [0.19013065, 0.80986935]])

In [246]:
y_pred[5]

1.0

In [291]:
#Getting the Metrics 
metric_preds = BinaryLabelDatasetMetric(data_val_pred, 
                            unprivileged_groups=unprivileged_groups, 
                            privileged_groups=privileged_groups)

ex_metric_preds = MetricTextExplainer(metric_preds)

print(ex_metric_preds.mean_difference())
print('\n')
print(ex_metric_preds.disparate_impact())


model_metric = ClassificationMetric(data_val, data_val_pred, 
                            unprivileged_groups=unprivileged_groups, 
                            privileged_groups=privileged_groups)

ex_model_metric= MetricTextExplainer(model_metric)
print('\n')
print(ex_model_metric.recall())
print('\n')
print(ex_model_metric.precision())
print('\n')
print(ex_model_metric.average_odds_difference())
print('\n')
print(ex_model_metric.equal_opportunity_difference())
print('\n')
print(ex_model_metric.theil_index())

Mean difference (mean label value on privileged instances - mean label value on unprivileged instances): -0.18825947645245145


Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.7894523326572008


True positive rate (TPR, recall, sensitivity = TP / (TP + FN)): 0.9977777777777778


Positive predictive value (PPV, precision = TP / (TP + FP)): 0.7230273752012882


Average odds difference (average of TPR difference and FPR difference, 0 = equality of odds): -0.13791230963088524


True positive rate difference (true positive rate on unprivileged instances - true positive rate on privileged instances): -0.0022664723976040246


Theil index (generalized entropy index with alpha = 1): 0.0535085341324013


In [260]:
scorecard(y_test, y_rw_pred)

The Accuracy score is 0.795.

The Precision score is 0.754.

The Recall score is 0.997.

      Confusion Matrix


,Actual True,Actual False
Predicted True,933,305
Predicted False,3,259


In [207]:
data_val.labels.shape

(1500, 1)

In [208]:
data_val_pred.labels.shape

(1500,)

## Debiasing through Preprocessing 

#### Reweighing

In [292]:
RW = Reweighing(unprivileged_groups=unprivileged_groups, 
               privileged_groups=privileged_groups)

#Splitting the Data
data_rw_train = RW.fit_transform(data_train)

data_rw_val = RW.fit_transform(data_val)

In [293]:
### Reweigh Metrics
metric_rw_train = BinaryLabelDatasetMetric(data_rw_train, 
                            unprivileged_groups=unprivileged_groups, 
                            privileged_groups=privileged_groups)
#Explainer 
ex_metric_rw_train = MetricTextExplainer(metric_rw_train)

print(ex_metric_rw_train.mean_difference())
print('\n')
print(ex_metric_rw_train.disparate_impact())

Mean difference (mean label value on privileged instances - mean label value on unprivileged instances): 0.0


Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 1.0


### Random Forest Classifier with Reweighing 

In [294]:
#Creating X,y
X_rw_train = data_rw_train.features
y_rw_train = data_rw_train.labels.ravel()
X_rw_test = data_rw_val.features
y_rw_test = data_rw_val.labels.ravel()

In [303]:
# Creating Random Forest Classifier
rfc_rw = RandomForestClassifier(n_estimators=1000, max_depth=5)
rfc_rw.fit(X_rw_train, y_rw_train, sample_weight=data_rw_train.instance_weights)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [304]:
#Creating Predicted Dataframe
y_rw_pred = rfc_rw.predict(X_test)
data_val_rw_pred = data_rw_val.copy()
data_val_rw_pred.labels = y_rw_pred

In [305]:
#Getting the Metrics 
display(Markdown("#### Model - with reweighing - dataset metrics"))


metric_rw_preds = BinaryLabelDatasetMetric(data_val_rw_pred, 
                            unprivileged_groups=unprivileged_groups, 
                            privileged_groups=privileged_groups)

ex_metric_rw_preds = MetricTextExplainer(metric_rw_preds)

print(ex_metric_rw_preds.mean_difference())
print('\n')
print(ex_metric_rw_preds.disparate_impact())

display(Markdown("#### Model - with reweighing - classification metrics"))
model_rw_metric = ClassificationMetric(data_rw_val, data_val_rw_pred, 
                            unprivileged_groups=unprivileged_groups, 
                            privileged_groups=privileged_groups)

ex_model_rw_metric= MetricTextExplainer(model_rw_metric)

print(ex_model_rw_metric.recall())
print(ex_model_rw_metric.precision())
print(ex_model_rw_metric.average_odds_difference())
print(ex_model_rw_metric.equal_opportunity_difference())
print(ex_model_rw_metric.theil_index())
print(ex_model_rw_metric.accuracy(privileged=True))
print(ex_model_rw_metric.accuracy(privileged=False))

#### Model - with reweighing - dataset metrics

Mean difference (mean label value on privileged instances - mean label value on unprivileged instances): -0.1107831421842288


Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.8736657919674279


#### Model - with reweighing - classification metrics

True positive rate (TPR, recall, sensitivity = TP / (TP + FN)): 0.9976387674707247
Positive predictive value (PPV, precision = TP / (TP + FP)): 0.7143138695057106
Average odds difference (average of TPR difference and FPR difference, 0 = equality of odds): -0.13791230963088524
True positive rate difference (true positive rate on unprivileged instances - true positive rate on privileged instances): -0.0022664723976040246
Theil index (generalized entropy index with alpha = 1): 0.0535085341324013
Classification accuracy on privileged instances: 0.7212167214866043
Classification accuracy on unprivileged instances: 0.8292800967937086


In [307]:
scorecard(y_rw_test, y_rw_pred)

The Accuracy score is 0.769.

The Precision score is 0.723.

The Recall score is 0.998.

      Confusion Matrix


,Actual True,Actual False
Predicted True,898,344
Predicted False,2,256


### Inprocessing with Adversarial Debiaising



In [266]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

In [267]:
##Creating Tensorflow Session - must not use Tensorflow 2.0 
sess = tf.Session()
db_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                            num_epochs  = 100,
                            batch_size = 100,
                            adversary_loss_weight = .1,     
                            debias=True,
                          sess=sess)

In [268]:
db_model.fit(data_train) 

epoch 0; iter: 0; batch classifier loss: 1.122157; batch adversarial loss: 0.791225
epoch 1; iter: 0; batch classifier loss: 0.534451; batch adversarial loss: 0.836398
epoch 2; iter: 0; batch classifier loss: 0.607169; batch adversarial loss: 0.815159
epoch 3; iter: 0; batch classifier loss: 0.507684; batch adversarial loss: 0.806940
epoch 4; iter: 0; batch classifier loss: 0.471816; batch adversarial loss: 0.729057
epoch 5; iter: 0; batch classifier loss: 0.701160; batch adversarial loss: 0.718264
epoch 6; iter: 0; batch classifier loss: 0.458773; batch adversarial loss: 0.700663
epoch 7; iter: 0; batch classifier loss: 0.424822; batch adversarial loss: 0.692936
epoch 8; iter: 0; batch classifier loss: 0.469862; batch adversarial loss: 0.661269
epoch 9; iter: 0; batch classifier loss: 0.470518; batch adversarial loss: 0.689723
epoch 10; iter: 0; batch classifier loss: 0.507576; batch adversarial loss: 0.647690
epoch 11; iter: 0; batch classifier loss: 0.437040; batch adversarial loss:

In [269]:
dataset_debiasing_train = db_model.predict(data_train)
dataset_debiasing_test = db_model.predict(data_val)

In [270]:

# Metrics for the dataset from model with debiasing
display(Markdown("#### Model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_train.mean_difference())

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_test.mean_difference())



display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(data_val, 
                                                 dataset_debiasing_test,
                                                    unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test.theil_index())
                                                        
                                                        

#### Model - with debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.064034
Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.026610


#### Model - with debiasing - classification metrics

Test set: Classification accuracy = 0.747333
Test set: Balanced classification accuracy = 0.719324
Test set: Disparate impact = 1.040472
Test set: Equal opportunity difference = 0.237269
Test set: Average odds difference = 0.073446
Test set: Theil_index = 0.154229


In [271]:
scorecard(data_val.labels, dataset_debiasing_test.labels)

The Accuracy score is 0.747.

The Precision score is 0.778.

The Recall score is 0.832.

      Confusion Matrix


,Actual True,Actual False
Predicted True,779,222
Predicted False,157,342


### Postprocessing with Equalized Odds Postprocessing

In [272]:

# Learn parameters to equalize odds and apply to create a new dataset
cpp = CalibratedEqOddsPostprocessing(privileged_groups = privileged_groups,
                                     unprivileged_groups = unprivileged_groups,
                                     cost_constraint='fpr')
cpp = cpp.fit_predict(data_val, data_val_pred)

In [273]:
cpp_metric = ClassificationMetric(data_val, cpp,
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

In [274]:
print(cpp_metric.generalized_false_positive_rate())

0.0


In [275]:
print(cpp_metric.generalized_false_negative_rate())

0.0


In [276]:
scorecard(data_val.labels, cpp.labels)

The Accuracy score is 1.000.

The Precision score is 1.000.

The Recall score is 1.000.

      Confusion Matrix


,Actual True,Actual False
Predicted True,936,0
Predicted False,0,564
